In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pickle
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from time import time
import argparse
import pdb


import torch
from torch.utils.data import Dataset
import torch.nn.functional as F
import torch.nn as nn
from torch import Tensor
from torch.nn import Transformer
from torch.utils.data import Dataset
import torch.nn.functional as F
from torch.utils.data import DataLoader

chars = " ^$#%()+-./0123456789=@ABCDEFGHIKLMNOPRSTVXYZ[\\]abcdefgilmnoprstuy"
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
VOCAB_SIZE=len(chars)
TGT_VOCAB_SIZE, SRC_VOCAB_SIZE = VOCAB_SIZE,VOCAB_SIZE
MAX_SEQ_LEN=160
PAD_IDX, BOS_IDX, EOS_IDX = 0,1,2

file = open("/home/arvid/data/USTPO_paper_5x/USTPO_5x_parsed.pickle",'rb')
data = pickle.load(file)



class ReactionDataset(Dataset):

    def __init__(self, data, split, maxlen=160, rep=" ^$#%()+-./0123456789=@ABCDEFGHIKLMNOPRSTVXYZ[\\]abcdefgilmnoprstuy"):
        self.split = split
        self.data = data[self.split]
        self.maxlen = maxlen
        self.rep = rep
        self.char_to_ix = { ch:i for i,ch in enumerate(rep) }
        self.ix_to_char = { i:ch for i,ch in enumerate(rep) }
        # Add augmentation methods here later
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        rs_smiles = self.data[index]["rs"]
        if rs_smiles[0] == ".":
            rs_smiles = rs_smiles[1:]
        ps_smiles = self.data[index]["ps"]
        
        
        rs_smiles = self.ix_to_char[BOS_IDX] + rs_smiles + self.ix_to_char[EOS_IDX] + (self.maxlen-len(rs_smiles))*" "
        ps_smiles = ps_smiles + (self.maxlen-len(ps_smiles)+2)*" "
        
        # Augment smiles here for train
        
        rs = torch.tensor([self.char_to_ix[char] for char in rs_smiles])
        ps = torch.tensor([self.char_to_ix[char] for char in ps_smiles])
        
        return {
            "rs": rs.to(dtype=torch.int64),
            "ps": ps.to(dtype=torch.int64)
            #'rs': F.one_hot(rs.to(dtype=torch.int64), num_classes=len(self.rep)),
            #'ps':  F.one_hot(ps.to(dtype=torch.int64), num_classes=len(self.rep))
        }

datasets = {}
dataloaders = {}
for split in ['train', 'eval']:
    datasets[split] = ReactionDataset(data=data,
                                   split=split)

    dataloaders[split] = DataLoader(datasets[split],
                                    batch_size=32,
                                    shuffle=(split != 'test'),
                                    num_workers=4,
                                    pin_memory=False)# Was True before.


def generate_square_subsequent_mask(sz):
        mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX)
    tgt_padding_mask = (tgt == PAD_IDX)
    tgt_padding_mask[tgt == EOS_IDX] = True
    return src_mask, tgt_mask, src_padding_mask.permute(1,0), tgt_padding_mask.permute(1,0)

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 200):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        token_embedding = token_embedding
        return self.dropout((token_embedding + self.pos_embedding[:token_embedding.size(0), :]))
    
    
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                    N_ENCODERS: int=6,
                    N_DECODERS: int=6,
                    EMBEDDING_SIZE: int=512,
                    N_HEADS: int=8,
                    SRC_VOCAB_SIZE: int=VOCAB_SIZE,
                    TGT_VOCAB_SIZE: int=VOCAB_SIZE,
                    DIM_FF: int=512,
                    DROPOUT: float=0.0):
        super(Seq2SeqTransformer, self).__init__()
        
        
        self.transformer = Transformer(d_model=EMBEDDING_SIZE,
                                       nhead=N_HEADS,
                                       num_encoder_layers=N_ENCODERS,
                                       num_decoder_layers=N_DECODERS,
                                       dim_feedforward=DIM_FF,
                                       dropout=DROPOUT)
        
        self.generator = nn.Linear(EMBEDDING_SIZE, TGT_VOCAB_SIZE)
        
        self.src_tok_emb = TokenEmbedding(SRC_VOCAB_SIZE, EMBEDDING_SIZE)
        self.tgt_tok_emb = TokenEmbedding(TGT_VOCAB_SIZE, EMBEDDING_SIZE)
        
        self.positional_encoding = PositionalEncoding(
            EMBEDDING_SIZE, dropout=DROPOUT)

    def forward(self,
                src: Tensor,
                tgt: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(tgt))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor, src_padding_mask: Tensor):
        return self.transformer.encoder(src=self.positional_encoding(
                            self.src_tok_emb(src)), mask=src_mask, src_key_padding_mask=src_padding_mask)

    def decode(self, tgt: Tensor, memory: Tensor, memory_key_padding_mask: Tensor):
        
        return self.transformer.decoder(tgt=self.positional_encoding(self.tgt_tok_emb(tgt)),
                                        memory=memory,
                                        memory_key_padding_mask=memory_key_padding_mask)
    
def loss_fn(logits, tgt, tgt_padding_mask):
    #Shift outs and labels one step
    logits = logits[:-1,:,:].permute(1,0,2)
    one_hot_targets = F.one_hot(tgt, num_classes=VOCAB_SIZE)[1:,:,:].permute(1,0,2)
    weights = (~tgt_padding_mask[:,:-1]).long().unsqueeze(-1).tile([1,1,TGT_VOCAB_SIZE])
    
    criterion = torch.nn.BCEWithLogitsLoss(reduction="none")
    loss = torch.sum(torch.mean(criterion(logits, one_hot_targets.float())*weights,axis=0),axis=(0,1))
    return loss




In [3]:
class CosineAnnealingWarmup():

    def __init__(self,
                 max_lr : float = 1e-3,
                 min_lr : float = 1e-4,
                 warmup_steps : int = 10,
                 base_lr: float = 1e-4,
                 gamma : float = 1,
                 last_epoch : int = 200
        ):
        
        self.base_max_lr = max_lr # first max learning rate
        self.max_lr = max_lr # max learning rate in the current cycle
        self.min_lr = min_lr # min learning rate
        self.warmup_steps = warmup_steps # warmup step size
        self.gamma = gamma # decrease rate of max learning rate by cycle
        self.base_lr = base_lr
        
        
    
    def get_lr(self, step):
        if step <= self.warmup_steps:
            return (self.max_lr - self.base_lr)*step / self.warmup_steps + self.base_lr
        elif step <= 100:
            return self.base_lr + (self.max_lr - self.base_lr) \
                    * (1+ math.cos(math.pi * (step - self.warmup_steps)/(step*1.5- self.warmup_steps))) / 2
    


def train_epoch(model, optimizer, split):
    if split == "train": model.train()
    else: model.eval()
        
    losses = 0
    for batch in dataloaders[split]: #tqdm(dataloaders[split], colour='WHITE'):
        src = batch["ps"].to(DEVICE).permute(1,0)
        tgt = batch["rs"].to(DEVICE).permute(1,0)


        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src=src, tgt=tgt)

        logits = model(src=src,tgt=tgt,src_mask=src_mask,
                            tgt_mask=tgt_mask,
                            src_padding_mask=src_padding_mask,
                            tgt_padding_mask=tgt_padding_mask,
                            memory_key_padding_mask=src_padding_mask)


        optimizer.zero_grad()

        loss = loss_fn(logits, tgt, tgt_padding_mask)
        if split=="train": loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(dataloaders[split])

def training_monitoring(model, optimizer, scheduler, n_epochs, args):
    
    model_dir = os.path.expanduser("~/models/transformers/")
    time()
    from datetime import datetime
    date_time = datetime.fromtimestamp(time())
    model_str = datetime.now().strftime("%Y-%m-%d-%H:%M:%S")
    os.mkdir(model_dir + model_str)
    os.mkdir(model_dir + model_str + "/logdir")
    print("tensorboard --logdir " + model_dir + model_str + "/logdir/ --port=6006")
    
    
    
    with open(model_dir + model_str + '/params.pickle', 'wb') as handle:
        pickle.dump(args, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    
    best_loss = 999999999
    running_losses = {'train': [], 'eval': []}    
        
    # Loop
    writer = SummaryWriter(model_dir + model_str + "/logdir/")
    
    for epoch in range(n_epochs):
        losses = {}
        print()
        
        lr = scheduler.get_lr(epoch)
        print("Beginning of epoch %i LR: %.4f"%(epoch, lr), end="")
        for g in optimizer.param_groups:
            g['lr'] = lr
        
        for split in ["train", "eval"]:
            t1 = time()
            print("..." + split, end="")
            #print("Epoch " + str(epoch) + " " + split + " progression:")
            epoch_loss = train_epoch(model=model, optimizer=optimizer, split=split)
            
            losses[split] = epoch_loss
            running_losses[split].append(epoch_loss)
            writer.add_scalar('Loss/'+split, epoch_loss, epoch+1)
            
            if split == 'eval' and epoch_loss < best_loss:
                    best_loss = epoch_loss
                    best_model_wts = model.state_dict()
                    torch.save(model.state_dict(),model_dir + model_str + "/weights")
            print("...finished in %.2f s! "%(time()-t1), end="")
        print()
        print("Epoch %i losses are; Train: %.2f // Eval: %.2f."%(epoch,losses["train"],losses["eval"]))
        
        

In [4]:
args= {"N_ENCODERS": [6, int],
                "N_DECODERS": [6, int],
                "EMBEDDING_SIZE": [512, int],
                "N_HEADS": [8, int],
                "DIM_FF": [512, int],
                "DROPOUT": [0.1, float],
                "LR": [1e-5, float],
                "EPOCHS": [200, int],
                "BATCH_SIZE": [32, int],
                "VOCAB_SIZE": [VOCAB_SIZE, int],
                "MAX_SEQ_LEN": [MAX_SEQ_LEN, int],
                "MAX_LR": [1e-3, float],
                "MIN_LR": [1e-4, float],
                "WARMUP_STEPS": [10, int],
                "BASE_LR": [1e-4, float],
                "GAMMA": [1.0, float],
                "device": [1, int],
                "checkpoint": ["2021-09-24-12:38:55", "2021-09-24-12:38:55"]}

for key in args:
    args[key] = args[key][0]
#pdb.set_trace()
print()
print("RUN PARAMS: ")
for a in args:
    print(str(a) + ": " + str(args[a]))

if args["device"] is not None: dev=args["device"]
else: dev="1"
global DEVICE
DEVICE = torch.device('cuda:'+str(dev) if torch.cuda.is_available() else 'cpu')
torch.torch.cuda.set_device(DEVICE)
print("Using device " + str(torch.torch.cuda.current_device()) + "/" + str(torch.cuda.device_count())
      +", name: " + str(torch.cuda.get_device_name(0)))

#torch.manual_seed(1337)

chemFormer = Seq2SeqTransformer(N_ENCODERS=args["N_ENCODERS"],
                                N_DECODERS=args["N_DECODERS"],
                                EMBEDDING_SIZE=args["EMBEDDING_SIZE"],
                                N_HEADS=args["N_HEADS"],
                                SRC_VOCAB_SIZE=SRC_VOCAB_SIZE,
                                TGT_VOCAB_SIZE=TGT_VOCAB_SIZE,
                                DIM_FF=args["DIM_FF"],
                                DROPOUT=args["DROPOUT"]).to(DEVICE)

if args["checkpoint"] is not None:
    models_dir = "/home/arvid/models/transformers/"
    chemFormer.load_state_dict(torch.load(models_dir + args["checkpoint"] + "/weights"))
    print("Successfully loaded checkpoint: " + models_dir + args["checkpoint"] + "/weights")   
  


RUN PARAMS: 
N_ENCODERS: 6
N_DECODERS: 6
EMBEDDING_SIZE: 512
N_HEADS: 8
DIM_FF: 512
DROPOUT: 0.1
LR: 1e-05
EPOCHS: 200
BATCH_SIZE: 32
VOCAB_SIZE: 66
MAX_SEQ_LEN: 160
MAX_LR: 0.001
MIN_LR: 0.0001
WARMUP_STEPS: 10
BASE_LR: 0.0001
GAMMA: 1.0
device: 1
checkpoint: 2021-09-24-12:38:55
Using device 1/16, name: Tesla V100-SXM3-32GB
Successfully loaded checkpoint: /home/arvid/models/transformers/2021-09-24-12:38:55/weights


In [5]:
j=0
inferred_list = []
for batch in dataloaders["eval"]:
    print(j)
    j += 1
    src = batch["ps"].to(DEVICE).permute(1,0)
    tgt_for_mask = batch["rs"].to(DEVICE).permute(1,0)
    src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src=src, tgt=tgt_for_mask)
    memory = chemFormer.encode(src=src, src_mask=src_mask, src_padding_mask=src_padding_mask)

    #tgt = torch.tensor(BOS_IDX).unsqueeze(-1).tile([1,32]).to(DEVICE)
    in_list = []

    tgt = torch.tensor(BOS_IDX).unsqueeze(-1).tile([1,32]).to(DEVICE)
    fins = {}
    i=0
    while True:
        i += 1
        out = chemFormer.decode(tgt=tgt, memory=memory, memory_key_padding_mask=src_padding_mask)
        probs = chemFormer.generator(out[-1,:,:])
        preds = torch.argmax(probs, axis=1)
        tgt = torch.cat((tgt,preds.unsqueeze(0)),axis=0)
        for ix in torch.where(preds==EOS_IDX)[0].cpu().numpy():
            if ix not in fins:
                fins[ix] = tgt[:,ix].cpu().numpy()
        if i == 163:
            break
    
    trues = ["".join([ix_to_char[ix] for ix in batch["rs"].numpy()[z,1:] if ix != PAD_IDX])[:-1] for z in range(32)]
    inferred_list.extend([[trues[k],"".join([ix_to_char[ix] for ix in v[1:-1]])] for k,v in fins.items()])
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

RuntimeError: shape '[-1, 256, 64]' is invalid for input of size 497664

In [6]:
inferred_list

[['BrCCCCC.c1(C=O)ncc(cc1)Oc1ccc2ccccc2n1', 'C'],
 ['[nH]1c2c(cccc2C)cc1.O(C(OC(=O)OC(C)(C)C)=O)C(C)(C)C', 'C'],
 ['C1=CCNC1.CC(C)(C)OC(=O)OC(=O)OC(C)(C)C', 'C'],
 ['O(CCCOc1ccc(F)cc1)c1ccc(cc1)C1CCN(CC1O)C(OC(C)(C)C)=O', 'C'],
 ['Cc1cc(Cl)cc(Cl)c1N.S=C(Cl)Cl', 'C'],
 ['O=C.CC1CN(CC(N1)C)C(OC(C)(C)C)=O', 'Cl'],
 ['c1(ccc(n1C1CCCCC1)-c1ccc(S(C)(=O)=O)cc1)C', 'Cc'],
 ['C1OC(OCC1(C)COC(c1ccc(O)cc1)=O)c1ccccc1', 'C=N'],
 ['O(c1c(OCCN2CCCCC2)cc(cc1)C1(CC(O)N(C1=O)c1cc(Cl)c(Cl)cc1)C)C', 'Cc1'],
 ['C(C)(C)(OC(CC(=O)c1cc(-n2c(COC3OCCCC3)cnn2)ccc1)=O)C.C1CC1N(c1cc(c(cc1Cl)N)NC(=O)OC(C)(C)C)C',
  'C[C'],
 ['c1(COc2ccc(C(NCCCCl)=O)cc2)ccccc1.c1c(CC2(CCNCC2)O)cccc1', 'C=N'],
 ['c1(c(N)cccc1)N.c1c(CCC(O)=O)cccc1', 'CC)'],
 ['C(CCCCCCC(O)/C=C/C=C/c1c(cccc1)CCCCCO)C', 'COCc'],
 ['O=C(C(c1ccc(cc1F)-c1ccc(OC)cc1)OC)NCc1ccc(C#N)cc1', 'CcCO'],
 ['c1c(-n2c3c(C(O)=O)cccc3cc2)cccc1Cl', 'C)CcCC1'],
 ['[C@@H]1(C)CN(c2c3ccc(F)cc3ccc2)CCN1CCC1c2ccc(cc2CCO1)CN', 'CCOCC=N'],
 ['N(C)(CCC(c1ncccc1)c1ccc(C(=O)OCC)cc

In [ ]:
for batch in dataloaders["train"]))
src = batch["ps"].to(DEVICE).permute(1,0)
tgt = batch["rs"].to(DEVICE).permute(1,0)
src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src=src, tgt=tgt)

print("decoder input:")
print("".join(["[" + ix_to_char[e]+str(int(f))+"],  " for e,f in zip(tgt[:,0].cpu().numpy(),tgt_padding_mask[0,:].cpu().numpy())]))


tgts = torch.argmax(F.one_hot(tgt, num_classes=VOCAB_SIZE)[1:,:,:].permute(1,0,2),axis=2)
loss_weights = (~tgt_padding_mask[:,:-1]).long().unsqueeze(-1).tile([1,1,TGT_VOCAB_SIZE])


print("".join(["[" + ix_to_char[e]+str(f)+"],  " for e,f in zip(tgts[0,:].cpu().numpy(),loss_weights[0,:,0].cpu().numpy())]))
#chars = [ix_to_char[e] for e in torch.argmax(F.one_hot(tgt, num_classes=VOCAB_SIZE)[1:,:,:].permute(1,0,2), axis=2)[0,:].cpu().numpy()]
#print("".join(chars))
#print("".join(["["+ix_to_char[e]+","+str(int(a))+"]" for e,a in zip(tgt[:,0].cpu().numpy(),tgt_padding_mask[0,:].cpu().numpy())]))
#weights = (~tgt_padding_mask[:,:-1]).long().unsqueeze(-1).tile([1,1,TGT_VOCAB_SIZE])
#weights = weights[0,:,0].cpu().numpy()
#print("".join(["["+str(a)+str(b)+"]," for a,b in zip(weights,chars)]))
